# Example

In [ ]:
from os.path import join
from scripts.util import MyLogger
main_folder = join('tmp', 'ep')
gpt_folder = join(main_folder, 'gpt41')
target_langs = sorted([
    'de', #'es', 'fr', 'it', 'nl', 'pt', 'el', 'da', 'sv', 'fi'
])
logfile_path = join(gpt_folder, 'gpt_log.jsonl')
logger = MyLogger(logfile=logfile_path)


In [ ]:

from scripts.translators import translate_document
from scripts.util import load_sents, triplet_align_sents
from scripts.data_management import EPManager
dm  = EPManager()
for lang in target_langs:
    src_sents, tgt_sents = dm.get_sentence_pairs(lang, 'en', num_of_sents=400)
    logger.add_dataset_info(name='ep', num_of_sents=400)
    try:
        translate_document(
                text=src_sents,
                src_lang=lang,
                tgt_lang='en',
                mt_folder=gpt_folder,
                translator='gpt4',
                logger=logger,
            )
    except Exception as e:
        logger.log_error(error=e, src_lang=lang, tgt_lang='en', translator='gpt4')
        print(str(e))
        continue
    mt_sents = load_sents(gpt_folder, lang, 'en')
    print(lang, len(mt_sents))
    triplet_align_sents(
        mt_sents=mt_sents,
        src_sents=src_sents,
        tgt_sents=tgt_sents,
        src_lang=lang,
        tgt_lang='en',
        folder_path=gpt_folder
    )

de 400


In [3]:
import os
from scripts.scoring import ResultProducer
l2f = {
    f.replace('.jsonl', ''): join(gpt_folder, f) for f in os.listdir(gpt_folder) if f.endswith('.jsonl') and not f.endswith('log.jsonl')
}
len(l2f)

1

In [4]:
rp = ResultProducer(lang2files=l2f)
rp.compute_results()
rp.display_results()

     Src       BLEU       chrF
0  de-en  37.718667  61.798119


In [9]:
import json
with open(logfile_path, 'r') as f:
    log_data = [json.loads(ln) for ln in f]

In [11]:
avg_in_toks = sum([o['in_toks'] for o in log_data]) / len(log_data)
avg_out_toks = sum([o['out_toks'] for o in log_data]) / len(log_data)
avg_in_tiktoks = sum([o['in_tiktoks'] for o in log_data]) / len(log_data)
avg_out_tiktoks = sum([o['out_tiktoks'] for o in log_data]) / len(log_data)